# Cryptographic Protocols

Transport Layer Security (TLS) and Pretty Good Privacy (PGP) are examples of cryptographic protocols that combine the cryptographic primitives that we learned about in previous sections.

# Transport Layer Security (TLS)

TLS Formerly known as Secure Sockets Layer (SSL) is a cryptographic protocol suite to secure data in transfer. It is widely used over internet with different protocols such as HTTP, FTP, SMTP, etc. Different cipher suites provides variety of security protection services based on the users' needs.

<img src="include/tls_http.png" width=800>

# Pretty Good Privacy (PGP)

 - A data encryption/decryption tool
 - Can be used to encrypt and authenticat email, files, etc.
 - Created by Phil Zimmermann in 1991
 - A practical hybrid system that uses symmetric and asymmetric crypto

<img src="include/PGP_diagram.png" width=500>

image source: wikipedia

## *<font color=" #6495ED">Exercise</font>*

 - Implement PGP to encrypt some content as shown in the diagram above. Use AES in GCM mode.

 - def enc_key(public_key, message)
 - def dec_key(private_key, ciphertext)
 - def enc_msg(key, iv, msg)
 - def dec_msg(key, iv, tag, ciphertext)

## *<font color=" #b74138">Solution</font>*

In [1]:
import os
from cryptography.hazmat.backends import default_backend
from cryptography.hazmat.primitives.ciphers import Cipher, algorithms, modes
from cryptography.hazmat.primitives import hashes
from cryptography.hazmat.primitives.asymmetric import padding
from cryptography.hazmat.primitives.asymmetric import rsa

private_key = rsa.generate_private_key(
    public_exponent=65537,
    key_size=2048,
    backend=default_backend())

public_key = private_key.public_key()

def enc_key(public_key, message):
    ciphertext = public_key.encrypt(
        message,
        padding.OAEP(
            mgf=padding.MGF1(algorithm=hashes.SHA1()),
            algorithm=hashes.SHA1(),
            label=None))
    return ciphertext

def dec_key(private_key, ciphertext):
    plaintext = private_key.decrypt(
        ciphertext,
        padding.OAEP(
            mgf=padding.MGF1(algorithm=hashes.SHA1()),
            algorithm=hashes.SHA1(),
            label=None))
    return plaintext

def enc_msg(key, iv, msg):
    cipher = Cipher(algorithms.AES(key), modes.GCM(iv), backend=default_backend())
    encryptor = cipher.encryptor()
    cipher_text = encryptor.update(msg) + encryptor.finalize()
    return cipher_text, encryptor.tag

def dec_msg(key, iv, tag, ciphertext):
    cipher = Cipher(algorithms.AES(key), modes.GCM(iv, tag), backend=default_backend())
    decryptor = cipher.decryptor()
    plaintext = decryptor.update(ciphertext) + decryptor.finalize()
    return plaintext

In [2]:
k1 = os.urandom(16)
iv = os.urandom(16)
msg = b"O'Reilly Applied Cryptography with Python 2019"

cipher, tag = enc_msg(k1, iv, msg)
encrypted_key = enc_key(public_key, k1)

decrypted_key = dec_key(private_key, encrypted_key)
plaintext = dec_msg(decrypted_key, iv, tag, cipher)

In [3]:
print(plaintext)

b"O'Reilly Applied Cryptography with Python 2019"
